# Tracking Sparse Points with Lucas-Kanade method

We'll select couple of points and track them in the video, from frame to frame.


In [1]:
import numpy as np
import cv2

In [2]:
corner_track_params = dict(maxCorners=10, qualityLevel=0.3, minDistance=7, blockSize=7)

In [4]:
# in the first frame of the video we'll detect corners and then track them
# Window Size:
#    - smaller window => more sensitive to noise, we might miss larger/faster motions
#    - larger windows can catch faster motions but might miss smaller motions
#
# maxLevel defines the level in the image pyramid that will be used. 0 = original image, 1 = 1/2 resolution, 2 = 1/4 resolution etc...
# [Pyramid (image processing) - Wikipedia](https://en.wikipedia.org/wiki/Pyramid_(image_processing))
# maxLevel allows us to find optical flow at various resolutions of image
#
# count (maximum number of iterations)=10; More iterations means more exhaustive search for the points in the current frame.
# EPS (epsilon)=0.03; smaller eps means finishing earlier; exchsnghing speed vs accuracy of tracking
# 
lucas_kanade_params = dict(winSize=(200, 200), maxLevel=2, criteria=(cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03))

In [29]:
# grab an image from camera

# cap = cv2.VideoCapture(0)

videoCaptureApi = cv2.CAP_ANY # autodetect default API
cap = cv2.VideoCapture("/dev/video2", videoCaptureApi)

if not cap.isOpened():
    raise RuntimeError("ERROR! Unable to open camera")

ret, prev_frame = cap.read() # read frame
prev_gray = cv2.cvtColor(prev_frame, cv2.COLOR_BGR2GRAY)

# define points which we want to track
prevPts = cv2.goodFeaturesToTrack(prev_gray, mask=None, **corner_track_params)

mask = np.zeros_like(prev_frame)

while True:
    ret, frame = cap.read()
    frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    nextPts, status, err = cv2.calcOpticalFlowPyrLK(prev_gray, frame_gray, prevPts, None, **lucas_kanade_params)
    
    # status is a vector; its elements are set to 1 if the flow for the corresponding feature has been found; otherwise it's set to 0.
    good_new = nextPts[status==1]
    good_prev = prevPts[status==1]
    for i, (new, prev) in enumerate(zip(good_new, good_prev)):
        # ravel() flattens 2d array into a 1d array
        x_new, y_new = new.ravel()  
        x_prev, y_prev = prev.ravel()
        
        mask = cv2.line(mask, (x_new, y_new), (x_prev, y_prev), (0, 255, 0), 3)
        frame = cv2.circle(frame, (x_new, y_new), 8, (0, 0, 255), -1)
    img = cv2.add(frame, mask)
    cv2.imshow('tracking', img)
    
    key = cv2.waitKey(30) & 0xFF
    if key == 27:
        break
        
    prev_gray = frame_gray.copy()
    prevPts = good_new.reshape(-1, 1, 2)
    
cv2.destroyAllWindows()
cap.release()